In [1]:
%pylab inline
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split



Populating the interactive namespace from numpy and matplotlib


/home/saket/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saket/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saket/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saket/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saket/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
normal_df = pd.read_table('/Z/personal-folders/interns/saket/histopath_data/patches_dataframe/training/normal/master_df.tsv')
normal_df['slide_type'] = 'normal'
normal_df['is_tumor'] = False

tumor_df = pd.read_table('/Z/personal-folders/interns/saket/histopath_data/patches_dataframe/training/tumor/master_df.tsv')
tumor_df['slide_type'] = 'tumor'
master_df = pd.concat([normal_df, tumor_df])

/home/saket/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



# Strategy

We will work with 200k patches for training and 30k patches for validation. The procedure is as:

1. Generate 'normal' or 'tumor' with probability = 0.5
2. Select a sample belonging to that category
   If tumor : Sample x tumor patches from the tumor such that, x/(total_tumor_patches) = 

In [3]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(tumor_df, tumor_df['is_tumor']):
    train_samples = tumor_df.loc[train_index]
    validation_samples = tumor_df.loc[test_index]

In [4]:
len(train_samples[train_samples.is_tumor==True].index)

235883

In [5]:
normal_slides_that_are_tumors = ['normal_086', 'normal_144']
# The following slides should not be used for generating 
# normal patches
# See "Detecting Cancer Metastases on Gigapixel Pathology Images"

tumor_slides_with_nonexhaustive_tumor_annotation = ['tumor_{}'.format(x) for x in ['018', '020', '029',
                                                                                   '033', '044', '046', 
                                                                                   '051', '054', '055', 
                                                                                   '079', '092', '095', 
                                                                                   '010', '025', '034', 
                                                                                   '056', '067', '085', 
                                                                                   '110']]


In [6]:
df = tumor_df['uid'].groupby(tumor_df['is_tumor']).value_counts().rename('counts')#.reset_index(drop=True)#.groupby('is_tumor')
df = pd.DataFrame(df).reset_index()

In [7]:
all_tumor_slides = sorted(tumor_df['uid'].unique())
all_normal_slides = sorted(normal_df['uid'].unique())

# Remove normal slides that are tumors
all_normal_slides = [x for x in all_normal_slides if x not in normal_slides_that_are_tumors]

In [8]:
# Keep 10 slides for validation 
tumor_train_slides, tumor_validate_slides = train_test_split(all_tumor_slides, test_size=0.08, random_state=42)
normal_train_slides, normal_validate_slides = train_test_split(all_normal_slides, test_size=0.08, random_state=42)
tumor_train_slides = sorted(tumor_train_slides)
tumor_validate_slides = sorted(tumor_validate_slides)
normal_train_slides = sorted(normal_train_slides)
normal_validate_slides = sorted(normal_validate_slides)

In [9]:
tumor_validate_slides

['tumor_005',
 'tumor_011',
 'tumor_031',
 'tumor_046',
 'tumor_065',
 'tumor_069',
 'tumor_079',
 'tumor_085',
 'tumor_097']

# How many patches in train/test dataset?

In [10]:
tumor_train_df = tumor_df[tumor_df.uid.isin(tumor_train_slides)]
tumor_validate_df = tumor_df[tumor_df.uid.isin(tumor_validate_slides)]

tumor_train_df.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/tumor_train_df.tsv',
                                     sep='\t',                                    
                                     index=False,
                                     header=True)
tumor_validate_df.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/tumor_validate_df.tsv',
                                     sep='\t',                                    
                                     index=False,
                                     header=True)


In [11]:
tumor_train_df_summary = tumor_train_df['uid'].groupby(tumor_train_df['is_tumor']).value_counts().groupby('is_tumor').describe()
tumor_train_df_summary['total'] = tumor_train_df_summary['mean'] * tumor_train_df_summary['count']
tumor_train_df_summary['cumprop']= tumor_train_df_summary.total.cumsum()/tumor_train_df_summary['total'].sum()
total_tumor_train_patches = tumor_train_df_summary.loc[True]['total']
tumor_train_df_summary.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/tumor_train_df_summary.tsv',
                                            sep='\t',
                                            index=False,
                                            header=True)
tumor_train_df_summary

,count,mean,std,min,25%,50%,75%,max,total,cumprop
is_tumor,,,,,,,,,,
False,102.0,27396.921569,16290.248338,4656.0,15404.25,23995.0,36917.75,108210.0,2794486.0,0.91299
True,102.0,2610.990196,5893.431154,6.0,64.75,491.0,1820.75,36705.0,266321.0,1.00000


In [12]:
total_tumor_train_patches

266321.0

In [13]:
tumor_validate_df_summary = tumor_validate_df['uid'].groupby(tumor_validate_df['is_tumor']).value_counts().groupby('is_tumor').describe()
tumor_validate_df_summary['total'] = tumor_validate_df_summary['mean'] * tumor_validate_df_summary['count']
tumor_validate_df_summary['cumprop']= tumor_validate_df_summary.total.cumsum()/tumor_validate_df_summary['total'].sum()
total_tumor_validate_patches = tumor_validate_df_summary.loc[True]['total'] 
tumor_validate_df_summary.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/tumor_validate_df_summary.tsv',
                                               sep='\t',
                                               index=False,
                                               header=True)

tumor_validate_df_summary

,count,mean,std,min,25%,50%,75%,max,total,cumprop
is_tumor,,,,,,,,,,
False,9.0,17634.777778,7928.999429,8140.0,13107.0,14869.0,20619.0,34265.0,158713.0,0.847618
True,9.0,3170.333333,4639.188884,65.0,156.0,351.0,3687.0,13296.0,28533.0,1.000000


In [14]:
normal_train_df = normal_df[normal_df.uid.isin(normal_train_slides)]
normal_validate_df = normal_df[normal_df.uid.isin(normal_validate_slides)]

normal_train_df.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_train_df.tsv',
                                     sep='\t',                                    
                                     index=False,
                                     header=True)
normal_validate_df.reset_index().to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_validate_df.tsv',
                                     sep='\t',                                    
                                     index=False,
                                     header=True)

In [15]:
normal_train_df_summary = normal_train_df['uid'].value_counts().describe()
normal_train_df_summary['total'] = normal_train_df_summary['mean'] * normal_train_df_summary['count']
normal_train_df_summary = pd.DataFrame(normal_train_df_summary).T
total_normal_train_patches = normal_train_df_summary['total']
normal_train_df_summary.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_train_df_summary.tsv',
                                 sep='\t',
                                 index=False,
                                 header=True)

normal_train_df_summary

,count,mean,std,min,25%,50%,75%,max,total
uid,145.0,24312.482759,16195.726265,2272.0,12932.0,21031.0,31868.0,80995.0,3525310.0


In [16]:
normal_validate_df_summary = normal_validate_df['uid'].value_counts().describe()
normal_validate_df_summary['total'] = normal_validate_df_summary['mean'] * normal_validate_df_summary['count']
normal_validate_df_summary = pd.DataFrame(normal_validate_df_summary).T
total_normal_validate_patches = normal_validate_df_summary['total']
normal_validate_df_summary.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_validate_df_summary.tsv',
                                 sep='\t',
                                 index=False,
                                 header=True)
normal_validate_df_summary


,count,mean,std,min,25%,50%,75%,max,total
uid,13.0,20053.923077,13488.247437,2966.0,8735.0,16560.0,30871.0,41933.0,260701.0


So we have a lot of normal patches and hence that will require subsampling.
We will subsample so that we subsample as many normal patches from tumor samples such that they match up
the counts of tumor patches (close to 225k) 
We start off training with an equal number of normal patches. Then given the tumor samples, we generate half of these normal patches from the valid tumor samples and the other half from the actual normal samples.







In [17]:
normal_train_df_finalized = pd.DataFrame()

relevant_tumor_normal_patches = tumor_train_df[(tumor_train_df.is_tumor==False) & (~tumor_train_df.uid.isin(tumor_slides_with_nonexhaustive_tumor_annotation))]

patch_frac = len(relevant_tumor_normal_patches.index)/(total_tumor_train_patches/2)
patch_frac = 1/patch_frac
for seed, tumor_uid in enumerate(tumor_train_slides):
    if tumor_uid in tumor_slides_with_nonexhaustive_tumor_annotation:
        # Should not use these slides for generating normal patches
        continue
    # How many normal patches does this uid have?
    tumor_train_uid_df = tumor_train_df[tumor_train_df.uid==tumor_uid]
    tumor_train_uid_df_normal = tumor_train_uid_df[tumor_train_uid_df.is_tumor==False]
    
    normal_patches_in_uid = len(tumor_train_uid_df_normal.index)
    tumor_patches_in_uid = len(tumor_train_uid_df[tumor_train_uid_df.is_tumor==True].index)
    
    
    # subsample these proprotion of 
    selected_patches = tumor_train_uid_df_normal.sample(frac=patch_frac, random_state=seed+42)
    normal_train_df_finalized = pd.concat([normal_train_df_finalized, selected_patches])

In [18]:
patch_frac

0.057552630680415974

In [19]:
len(normal_train_df_finalized.index)

133160

In [20]:
# Select the remaining half from the actual normal slides.
patch_frac = len(normal_train_df.index)/(total_tumor_train_patches/2)
patch_frac = 1/patch_frac

for seed, normal_uid in enumerate(normal_train_slides):
    
    # How many normal patches does this uid have?
    normal_train_uid_df = normal_train_df[normal_train_df.uid==normal_uid]
    normal_patches_in_uid = len(normal_train_uid_df.index)
    
    # subsample these proprotion of 
    selected_patches = normal_train_uid_df.sample(frac=patch_frac, random_state=seed+42)
    normal_train_df_finalized = pd.concat([normal_train_df_finalized, selected_patches])

/home/saket/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [21]:
len(normal_train_df_finalized.index)

266317

In [22]:
total_tumor_train_patches

266321.0

In [23]:
total_tumor_validate_patches

28533.0

In [24]:
# Do the same thing for the validation
normal_validate_df_finalized = pd.DataFrame()

relevant_tumor_normal_patches = tumor_validate_df[(tumor_validate_df.is_tumor==False) & (~tumor_validate_df.uid.isin(tumor_slides_with_nonexhaustive_tumor_annotation))]

patch_frac = len(relevant_tumor_normal_patches.index)/(total_tumor_validate_patches/2)
patch_frac = 1/patch_frac
for seed, tumor_uid in enumerate(tumor_validate_slides):
    if tumor_uid in tumor_slides_with_nonexhaustive_tumor_annotation:
        # Should not use these slides for generating normal patches
        continue
    # How many normal patches does this uid have?
    tumor_validate_uid_df = tumor_validate_df[tumor_validate_df.uid==tumor_uid]
    tumor_validate_uid_df_normal = tumor_validate_uid_df[tumor_validate_uid_df.is_tumor==False]
    
    normal_patches_in_uid = len(tumor_validate_uid_df_normal.index)
    tumor_patches_in_uid = len(tumor_validate_uid_df[tumor_validate_uid_df.is_tumor==True].index)
    
    
    # subsample these proprotion of 
    selected_patches = tumor_validate_uid_df_normal.sample(frac=patch_frac, random_state=seed+42)
    normal_validate_df_finalized = pd.concat([normal_validate_df_finalized, selected_patches])

In [25]:
print(len(normal_validate_df_finalized.index))

14265


In [26]:
patch_frac = len(relevant_tumor_normal_patches.index)/(total_tumor_validate_patches/2)
patch_frac = 1/patch_frac
patch_frac

0.15786241465924555

In [27]:
total_tumor_validate_patches

28533.0

In [28]:
# Select the remaining half from the actual normal slides.
patch_frac = len(normal_validate_df.index)/(total_tumor_validate_patches/2)
patch_frac = 1/patch_frac

for seed, normal_uid in enumerate(normal_validate_slides):
    
    # How many normal patches does this uid have?
    normal_validate_uid_df = normal_validate_df[normal_validate_df.uid==normal_uid]
    normal_patches_in_uid = len(normal_validate_uid_df.index)
    
    # subsample these proprotion of 
    selected_patches = normal_validate_uid_df.sample(frac=patch_frac, random_state=seed+42)
    normal_validate_df_finalized = pd.concat([normal_validate_df_finalized, selected_patches])

/home/saket/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [29]:
print(len(normal_validate_df_finalized.index))

28531


In [30]:
print(len(tumor_validate_df.index))

187246


In [31]:

normal_train_df_finalized.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_train_df_finalized.tsv',
                                 sep='\t',
                                 index=False,
                                 header=True)


normal_validate_df_finalized.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/normal_validate_df_finalized.tsv',
                                    sep='\t',
                                    index=False,
                                    header=True)

In [32]:
# Let's just merge the datasets for training
tumor_train_df_tumor = tumor_train_df[tumor_train_df.is_tumor==True]
normal_train_df_finalized['is_tumor'] = False

tumor_validate_df_tumor = tumor_validate_df[tumor_validate_df.is_tumor==True]
normal_validate_df_finalized['is_tumor'] = False

train_df = pd.concat([tumor_train_df_tumor, normal_train_df_finalized])
# Shuffle before saving so that we do not have things together
train_df = train_df.sample(frac=1, random_state=42)

validate_df = pd.concat([tumor_validate_df_tumor, normal_validate_df_finalized])
# Shuffle before saving so that we do not have things together
validate_df = validate_df.sample(frac=1, random_state=42)


train_df.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/train_df.tsv',
                sep='\t',
                index=False,
                header=True)

validate_df.to_csv('/Z/personal-folders/interns/saket/github/pywsi/data/patch_df/validate_df.tsv',
                   sep='\t',
                   index=False,
                   header=True)


/home/saket/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

/home/saket/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [33]:
len(train_df.index)

532638

In [34]:
len(tumor_validate_df.index)

187246

In [35]:
len(normal_validate_df_finalized.index)

28531

In [36]:
patch_frac = len(normal_validate_df.index)/(total_tumor_validate_patches/2)
patch_frac = 1/patch_frac
patch_frac

0.05472361057303194

In [37]:
len(validate_df.index)

57064